## Segment Anything Model (SAM) - EELS: live workflow
- Acquire an image using HAADF detector
- Feed HAADF image in SAM pipeline:
        - get particles with segmented masks
        - find center point of each particle
- Acquire eels detector signals at center of each particle
#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 2nd May 2025
#### edited - 
   

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
from stemOrchestrator.logging_config   import setup_logging
import matplotlib.pyplot as plt
import logging
plot = plt
from typing import Dict
import os

In [ ]:
########SAM part ********************************************************************************************************

from stemOrchestrator.MLlayer.MLlayerSAM import setup_device, download_sam_model, initialize_sam_model, preprocess_image, generate_and_save_masks, create_normalized_particle_positions, display_image_with_masks, display_image_with_labels, extract_mask_contours, generate_mask_colors, visualize_masks_with_boundaries, extract_particle_data, print_boundary_points_info, plot_centroids, sample_particle_positions, plot_sampled_positions, create_visualization_with_masks
import pickle
import numpy as np
from numpy.typing import NDArray 
from typing import List, Dict, Union


def run_sam(image_data: np.ndarray, path_folder: str) -> Union[List, Dict]:
    """Main function to run SAM segmentation pipeline."""
    device = setup_device()
    
    model_type = "vit_b"  # Options: 'vit_b', 'vit_l', 'vit_h'
    checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
    checkpoint_path = "sam_vit_b_01ec64.pth"
    download_sam_model(model_type, checkpoint_url, checkpoint_path)
    sam, mask_generator = initialize_sam_model(model_type, checkpoint_path, device)
    img_np = preprocess_image(image_data)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img_np)
    plt.title("Original Image")
    plt.axis('off')
    plt.show()
    
    # Generate and visualize masks
    masks_path = f'{path_folder}/masks_Au_online.pkl'
    masks = generate_and_save_masks(mask_generator, img_np, masks_path)
    visual_image, centroids = create_visualization_with_masks(img_np, masks)
    display_image_with_masks(visual_image, "Image with Segmentation Masks")
    display_image_with_labels(visual_image, centroids, "Image with Segmentation Masks and Labels")
    
    mask_contours = extract_mask_contours(masks)
    mask_colors = generate_mask_colors(len(masks))
    boundaries_path = f"{path_folder}/Segmentation Masks with Boundaries and Centroids.png"
    visualize_masks_with_boundaries(visual_image, centroids, mask_contours, mask_colors, boundaries_path)
    particles = extract_particle_data(masks)
    # Save particle data
    with open(f'{path_folder}/particles.pkl', 'wb') as f:
        pickle.dump(particles, f)
    
    print_boundary_points_info(particles)
    centroids_array = np.array(centroids)
    plot_centroids(centroids_array, img_np)
    positions_sampled = sample_particle_positions(particles, img_np)
    plot_sampled_positions(positions_sampled, img_np, len(centroids))
    each_particle_position = create_normalized_particle_positions(particles, img_np.shape[:2])
    with open(f'{path_folder}/sampled_boundary_pts_particles.pkl', 'wb') as f: # Save normalized particle positions
        pickle.dump(each_particle_position, f)
    
    all_particle_keys = each_particle_position.keys()


    
    print("Processing complete!")
    return all_particle_keys, each_particle_position

##########****************************************************************************************************************************


In [ ]:
def main(config :Dict) -> None:
    ip = config["ip"]
    port = config["port"]
    haadf_exposure = config["haadf_exposure"]
    haadf_resolution = config["haadf_resolution"]
    out_path = config["out_path"]
    setup_logging(out_path=out_path)
    

    microscope_tf = TemMicroscopeClient()
    microscope_tf.connect(ip, port = port)# 7521 on velox  computer
    # microscope_dm = DMtwin()
    # query state:

    global tf_acquisition
    tf_acquisition = TFacquisition(microscope=microscope_tf, offline=True)
    # dm_acquisition = DMacquisition(microscope=microscope_dm, offline=True)
    dm_acquisition  = DMtwin()



    # Get haadf
    haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(exposure = haadf_exposure, resolution=haadf_resolution)

    HAADF_tiff_to_png(haadf_tiff_name)
    logging.info("END acquisition.")
    
    all_particle_keys, each_particle_position = run_sam(haadf_np_array, out_path)
    
    

    def acquire_and_plot_combined(image_data, particle_key, particle_dict, eels_exposure, eels_offset):
        """Acquire EDS spectrum, CETA image, and plot all three components in a single figure."""
        
        
        # lets do just eels at boundary points
        print(f"eels for particle{particle_key}")
        # Ensure the directory exists
        directory = f'{out_path}/particle{particle_key}'
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Get EELS spectrum
        tf_acquisition.unblank_beam()
        # array_server.acquire_camera(exposure = eels_exposure)
        dm_acquisition.acquire_camera(exposure = eels_exposure)
        tf_acquisition.blank_beam() # blank the beam
        # array_list, shape, dtype = array_server.get_eels()# 0.15 is the scaling factor, dipersion
        array_list, shape, dtype  = dm_acquisition.get_eels()
        spec_eels = np.array(array_list, dtype=dtype).reshape(shape)
        # np.savez("data/eels_.npz", spec = array)

        
        
        
        
        # Get the current beam position
        position = tf_acquisition.query_paused_beam_positon()
        x = position.x
        y = position.y
        formatted_position = f"({x:.2g}, {y:.2g})"

        # Create a figure with three subplots
        fig, axs = plt.subplots(1, 2, figsize=(18, 6))

        # Plot the acquired image data
        axs[0].imshow(image_data, cmap='gray')
        axs[0].set_title('Acquired Image')
        axs[0].set_axis_off()  # Hide axes for the image plot
        axs[0].scatter(x * image_data.shape[0], y * image_data.shape[1], c='r', s=100, marker='x', label=f"Position: {formatted_position}")
        
        axs[1].plot(np.array(range(len(spec_eels))) - eels_offset, spec_eels)# -25 is the offset
        axs[1].set_title('Acquired EELS Spectrum')
        axs[1].set_xlabel("Energy loss (eV)")
        axs[1].set_ylabel('Intensity Counts')
        
        
        # axs[3].legend()

        # Adjust layout and display
        plt.tight_layout()
        plt.savefig(f'{out_path}/particle{particle_key}/haadf_eels_at_centroid of particle{particle_key}_{formatted_position}.png', dpi = 300)
        plt.close()


        # save the ceta and spectrum data
        # np.save(f'{path_folder}/ceta image at position {formatted_position}.npy', ceta_image_data)# ceta image save
        # np.save(f'{path_folder}/EDX_spectrum at position {formatted_position}.npy', spec)# eds spectrum save
        np.save(f'{out_path}/particle{particle_key}/eels at centroid of particle{particle_key}_{formatted_position}.npy', spec_eels)# eels spectrum save
        # ceta_image.save(f'{path_folder}/ceta_raw_ at position {formatted_position}.tiff')


        import threading
        import time
        def move_beam(tf_acquisition, boundary_pt_array, eels_exposure):
            print("beam movement started")
            tf_acquisition.unblank_beam()

            for pts in boundary_pt_array:
                point = list(pts)
                tf_acquisition.move_paused_beam(point[0], point[1])
                position = tf_acquisition.query_paused_beam_positon()
                x = position.x
                y = position.y
                formatted_position = f"({x:.4g}, {y:.4g})"
                print(f"Beam positioned at: {formatted_position}")
                time.sleep(eels_exposure/len(boundary_pt_array)*0.8)# eels exposure per point
            print("Beam scanning complete.")
            tf_acquisition.blank_beam()

        def acquire_eels_data(dm_acquisition, eels_exposure):
            print("EELS acquisition started.")
            dm_acquisition.acquire_camera(exposure=eels_exposure)
            print("EELS acquisition complete.")

        boundary_pt_array = particle_dict[particle_key]["boundary_points"]

        beam_thread = threading.Thread(target=move_beam, args=(tf_acquisition, boundary_pt_array,  eels_exposure))
        beam_thread.start()

        # Run EELS acquisition in parallel
        acquire_eels_data(dm_acquisition, eels_exposure)
        tf_acquisition.blank_beam()
        # Wait for beam movement to finish
        beam_thread.join()### I think exposure should be high enough so all the points is covered

        # Once both tasks are done, retrieve and save the EELS data
        array_list, shape, dtype = dm_acquisition.get_eels()
        spec_eels = np.array(array_list, dtype=dtype).reshape(shape)
        
        plt.close()
        # Plot and save the single EELS spectrum
        plt.plot(np.array(range(len(spec_eels))) - eels_offset, spec_eels)
        plt.title('Acquired EELS Spectrum for All Boundary Points')
        plt.xlabel("Energy loss (eV)")
        plt.ylabel('Intensity Counts')
        plt.savefig(f"{out_path}/particle{particle_key}/eels_acquisition_combined_boundary_particle{particle_key}.png")
        plt.close()
        np.save(f'{out_path}/particle{particle_key}/eels_acquisition_combined_boundary_particle{particle_key}.npy', spec_eels)


    def run_acquisition_for_particle(image_data, particle_key, particle_dict, eels_exposure, eels_offset):
        """Run the entire acquisition process for a given position."""
        position = list(particle_dict[particle_key]["centroid"])
        tf_acquisition.move_paused_beam(position[0], position[1])

        acquire_and_plot_combined(image_data, particle_key, particle_dict, eels_exposure, eels_offset)

    spec_metadata_dict = {"eels_exposure_sec" : 0.05, "eels_offset": 0}

    eels_exposure = spec_metadata_dict["eels_exposure_sec"]
    eels_offset = spec_metadata_dict["eels_offset"]

    file_path = f'{out_path}/spectrometer_metadata_dict.pkl'
    # Save the dictionary to disk
    with open(file_path, 'wb') as file:
        pickle.dump(spec_metadata_dict, file)
    print(f"spectrometer_metadata_dictsaved to {file_path}")

    for particle_key in all_particle_keys:
        run_acquisition_for_particle(haadf_np_array,particle_key, each_particle_position, eels_exposure, eels_offset)# 25 seconds per reading: --> 1min 35 seconds for 4 points

    

In [ ]:
import os
import json
from pathlib import Path

# Initialize to None
ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")
uri_eels_ip = None
uri_eels_port = None

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF_sim")
            port = port or secret.get("port_TF_sim")
            uri_eels_ip = uri_eels_ip or secret.get("uri_eels_ip")
            uri_eels_port = uri_eels_port or secret.get("uri_eels_port")

if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")

try:
    port = int(port)
except ValueError:
    raise ValueError("Port must be an integer")

config = {
    "ip": ip,
    "port": port,
    "uri_eels_ip": uri_eels_ip,
    "uri_eels_port": uri_eels_port,
    "haadf_exposure": 40e-8,  # microseconds per pixel
    "haadf_resolution": 512,  # square
    "out_path": "."
}

main(config=config)